# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files
This cell is culled from Udacity provided template for the project

In [2]:
# checking the current working directory 
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(len(file_path_list)) # get the total number of files in the path

/home/ernest/Documents/data engineering /apache_cassandra_data_modeling
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables
Culled from the Udacity provided template

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in the csv file. Provided by Udacity
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## The CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra codes in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance in the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS ernest \
                    WITH REPLICATION = \
                    {'class': 'SimpleStrategy', 'replication_factor': 1}")
    
except Exception as e:
    print(e)


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('ernest')
except Exception as e:
    print(e)


### The following tables are created for specific queries in mind. This is because in Apache Cassandra, the database tables are modeled based on the expected queries. The queries are presented below. The test.ipynb also shows the specific queries of each table

### Create a song_library table
#### PARTITION KEY is sessionId. This implies that data is distributed using the sessionId
#### CLUSTERING COLUMN is made up of the itemInsession to ensure unique PRIMARY KEY

In [8]:
session.execute("drop table IF EXISTS song_library")
query = "CREATE TABLE IF NOT EXISTS song_library"
query = query + "(sessionId INT, \
                  itemInSession INT,\
                  artist TEXT, \
                  song TEXT, \
                  length DOUBLE, \
                  PRIMARY KEY(sessionId, itemInsession))"             
try:
    session.execute(query)
except Exception as e:
    print('Error Creating table', e)

                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_library(SessionId, itemInSession, artist, song, length )"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]) ))

### Create artist_library table
#### PARTITION KEY is made up of userId and sessionId, that means data is distributed using userId and sessionId.
#### CLUSTERING COLUMN consists of itemInSession
##### songs are sorted by itemInSession

In [10]:
session.execute("drop table IF EXISTS artist_library")
query = "CREATE TABLE IF NOT EXISTS artist_library"
query = query + "(userId INT, \
                 sessionId INT, \
                 itemInSession INT,\
                 artist TEXT, \
                 first TEXT, \
                 last TEXT,\
                 song TEXT,  \
                 PRIMARY KEY((userId, sessionId), itemInSession))"             
try:
    session.execute(query)
except Exception as e:
    print('Error Creating table\n', e)

                    
                    

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO artist_library( userId, SessionId, itemInSession, artist, first, last, song )"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]) , line[0], line[1], line[4],  line[9]))

### Create user table 
#### PARTITION KEY is song. That means data is distributed  across  the nodes using song
#### CLUSTERING COLUMN is userId to keep the PRIMARY KEY UNIQUE

In [12]:
session.execute("drop table IF EXISTS user")
query = "CREATE TABLE IF NOT EXISTS user"
query = query + "(song TEXT, \
                 userId INT, \
                 artist TEXT,\
                 first TEXT, last TEXT, \
                 PRIMARY KEY(song, userId))"             
try:
    session.execute(query)
except Exception as e:
    print('Error Creating table\n', e)

                    
                    

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user( song, userId, artist, first, last )"
        query = query + "VALUES( %s, %s, %s, %s, %s)"
        
        session.execute(query, (line[9], int(line[10]), line[0], line[1], line[4]) )

### The data modeling is verified using the following queries:

####  Query 1:  Get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

In [14]:
query = "SELECT artist, song, length FROM song_library WHERE sessionId = 338 and itemInSession = 4  "
try:
    rows = session.execute(query)
    for row in rows:
        print( "The artist is '{}'. The song is '{}'. And the length is '{}'".format( row.artist, row.song, row.length))
except Exception as e:
    print(e)

The artist is 'Faithless'. The song is 'Music Matters (Mark Knight Dub)'. And the length is '495.3073'


#### The artist is <font color=green > Faithless </font>
#### The song is <font color=green >  Music Matters (Mark Knight Dub) </font>
#### And the length is <font color=green >  495.3073 </font>

#### Query 2: Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
query = "SELECT artist, song, first, last FROM artist_library WHERE userId = 10 and itemInSession = 0 and sessionId = 182"
try:
    rows = session.execute(query)
    for row in rows:
        print("The artist is '{}'.\n The title of the song is '{}'.\n The user is '{} {}'".format(row[0], row[1], row[2], row[3]))
except Exception as e:
    print(e)


The artist is 'Down To The Bone'.
 The title of the song is 'Keep On Keepin' On'.
 The user is 'Sylvie Cruz'


#### The artist is <font color=green >"Down To The Bone"</font>
#### The title of the song is <font color=green >"Keep On Keepin' On "</font>
#### The user is <font color=green >"Sylvie Cruz"</font>

####  Query 3: Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [16]:
query = "SELECT first, last FROM user WHERE song = 'All Hands Against His Own'"
try:
    song = 'All Hands Against His Own'
    rows = session.execute(query)
    print("The following user(s) listened to the song, '{}':".format(song))
    for row in rows:
        print(row.first, row.last)
except Exception as e:
    print(e)


The following user(s) listened to the song, 'All Hands Against His Own':
Jacqueline Lynch
Tegan Levine
Sara Johnson


#### <font color= blue> The following user(s) listened to the song,</font>  <font color= red> 'All Hands Against His Own'</font> :
#### <font color= green>  Jacqueline Lynch</font> 
#### <font color= green> Tegan Levine</font> 
#### <font color= green> Sara Johnson</font> 

### Drop the tables before closing out the sessions

In [17]:
drop_song_library = "DROP TABLE IF EXISTS song_library"
drop_artist_library = "DROP TABLE IF EXISTS artist_library"
drop_user = "DROP TABLE IF EXISTS user"
tables = [drop_song_library, drop_artist_library, drop_user]

for t in tables:
    session.execute(t)

#### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()